Resultados esperados:
 - input: 
    - avg_runtime
    - input_complexity
 - output: 
    - p_cores


In [19]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [17]:
df = pd.read_csv("dataset/clean_data.csv")

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(16,8)))
model.add(tf.keras.layers.LSTM(2, activation="relu", dropout=0.2))
model.add(tf.keras.layers.Dense(1))
model.compile(optimizer='sgd', loss='mse')

model.summary()

model.fit(x, y, batch_size=32, epochs=10)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 2)                 88        
                                                                 
 dense_2 (Dense)             (None, 8)                 24        
                                                                 
Total params: 112 (448.00 Byte)
Trainable params: 112 (448.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
